수백 개의 뉴런 + 10개 이상의 층 ~ 심층 신경망은 여러 문제를 마주한다.

# 1. Gradient 소실과 폭주 문제
역전파 알고리즘을 이용한 오차 gradient 전파 시, 경사 하강법 단계에서 파라미터를 수정하게 된다. \
알고리즘이 하위층으로 진행될 수록 gradient가 점점 작아지거나 커지는 경우가 많다. \
이러한 경우 vanishing gradient 그레이디언트 소실 혹은 exploding gradient 그레이디언트 폭주 라고한다. \
이러한 불안정한 gradient는 층마다 학습 속도가 달라질 수 있다. \
불안정한 gradient 의 원인으로는 (로지스틱 시그모이드 활성화 함수) + (정규분포 이용한 가중치 초기화 방법) 의 조합이였다. (정규분포 : 평균이 0, 표준편차가 1) \
해당 원인으로 인해서 분산이 output layer -> input layer 로 갈수록 커졌다. 

더구나 로지스틱 함수의 평균이 0이 아닌 0.5라서 정규분포와 맞지 않다는 점도 있다.

- 불안정한 Gradient 문제 해결하기

## 1.1 글로럿과 He 초기화
글로럿의 조건
1. 각 층의 출력에 대한 분산 = 입력에 대한 분산 
2. 역방향에서 층을 통과하기 전, 후 Gradient 분산이 동일해야 한다.

글로럿의 방법 -> 세이비어 초기화 or 글로럿 초기화
1. 각 층의 연결 가중치를 글로럿 초기화 방식대로 무작위로 초기화한다.

글로럿 초기화 방식이란?
- 평균이 0, 분산이 $\sigma^2={1\over fan_{avg}}$ 인 정규분포
혹은
- $r=\sqrt{3\over fan_{avg}}$ 일 때 $-r$과 $+r$ 사이의 균등분포

초기화 전략과 활성화 함수의 조합은 아래와 같다.
- 글로럿 : 활성화 함수없음, 하이퍼볼릭 탄젠트, 로지스틱, 소프트맥스
- He : ReLU 함수와 그 변종들
- 르쿤 : SELU

## 1.2 수렴하지 않는 활성화 함수

활성화 함수가 시그모이드가 최선일 것이라 생각 -> ReLU의 특정 양숫값에 수렴하지 않는 장점 파악 -> dying ReLU 문제 발생 -> ReLU 변종인 LeakyReLU 사용

dying ReLU란?
- 일부 뉴런이 0값만 출력
- 큰 학습률을 이용하면 뉴런의 절반이 죽어있기도
- 입력의 가중치 합이 음수가 되면 -> 그레이디언트가 0이된다 -> 뉴런 죽음

#### LeakyReLU
$LeakyReLU_\alpha (z)=max(\alpha z,z)$ \
$\alpha$ 하이퍼파라미터가 leaky 정도(z<0 일때의 기울기) 를 결정. \
보통 0.01로 설정. -> LeakyReLU가 죽지 않도록 해줌

LeakyReLU의 성능 > ReLU의 성능    
코드 예시 :
```python
model = keras.models.Sequential([
    ...
    keras.layers.Dense(10, kernel_initializer="he_normal"),
    keras.layers.LeakyReLU(alpha=0.2),
    ...
])
```

#### ELU
ReLU 변종에 비하여 훈련 시간 감소, 성능 향상 \
z<0 일때, gradient가 0이 아니게 만든다. 죽은 뉴런을 형성하지 않도록 함. \
z<0 일때, $\alpha$로 ELU가 수렴할 값을 정의함, 보통 1로 설정. \
1로 설정시 z=0일때, 변동이 완만하여 경사 하강 속도를 높여준다. \
지수 함수를 사용하므로 계산이 느리다 - 훈련시 수렴 속도가 빨라 느린 계산을 커버해준다. -> test시 속도가 느리다.

#### SELU
ELU 활성화 변종 \
완전 연결층 + 은닉층 활성화 함수(SELU) -> 네트워크가 자기 정규화됨 (output이 mean 0, 표준편차 1 유지)    
-> gradient 소실, 폭주 문제 커버    

자기 정규화가 일어나려면?    
- input feature가 반드시 표준화(평균 0, 표준편차1)이어야 함
- 모든 은닉층 가중치=르쿤 정규분포 초기화 설정. kernel_initializer="lecun_normal"
- 일렬로 쌓은 층으로 구성

코드예시:
```python
layer=keras.layers.Dense(10, activation="selu", kernel_initializer="lecun_normal")
```

SELU > ELU > LeakyReLU > ReLU > tanh > Logistic 순으로 시도해보면 좋다.

실행 속도가 중요하다면 - LeakyReLU    
과대적합되었다면 - RReLU    
훈련 세트가 아주 크면 - PReLU    
라이브러리, 하드웨어 가속기들이 최적화되어있는, 속도가 중요하다면 - ReLU

## 1.3 배치 정규화
Gradient 소실과 폭주 문제를 해결하기위한 것    
각 층에서 활성화 함수를 통과하기 전 or 후에 연산을 하나 추가한다.    
활성화 함수 이후보다 이전에 배치 정규화 층을 추가하는게 좋다.    
예) input 정규화 연산 -> hidden layer(1. scale 조정, 2. scale 이동) -> ...

1st layer에 배치 정규화 추가 - data 표준화 안해도 됨 
정규화과정을 진행하는 알고리즘이라면 평균과 표준편차 정보가 있어야한다.    
미니배치 - input의 평균과 표준편차를 평가한다. 훈련하는 동안 입력을 배치 정규화, 스케일 조정과 이동이 진행된다.    
하지만 이러한 과정은 예측값이 있어야 평균과 표준편차 계산이 가능한데, test data의 경우 계산할 방법이 없다.    
해결법 : 훈련완료 후 train data 전체를 이용해 배치 정규화 층에서의 (입력 평균, 표준편차의 이동 평균)을 사용한다.    
역전파으로 - (출력 스케일 벡터, 출력 이동 벡터) , 지수 이동 평균으로 - (최종 입력 평균 벡터, 최종 입력 표준편차 벡터) 여기서 ,    
지수 이동 평균으로 구한 값들은 훈련 중 계속해서 추정되고 훈련이 끝나면 사용되는 것이다.

배치정규화는 규제와 같은 역할을 하여 다른 규제 기법의 필요성을 줄여준다.    
모델의 복잡도 증가 -> 가중치를 업데이트하여 커버 가능    

코드구현 예시: BatchNormalization 층 추가
```python
model=keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28,28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(10, activation="softmax")
])
```

배치정규화의 마지막 층에서는 최종 통계를 사용한다. -> BatchNormalization 클래스의 내부를 보면 ,     
call() 함수를 찾을 수 있다.    
만약 훈련과 테스트에 다르게 동작하는 사용자 정의 층을 만들고자 한다면 call(training=...) training 변수를 정의해줘야 한다.

그리고 아래 코드를 이용하여 각 배치 정규화 층의 파라미터를 살펴볼 수 있다.    
아래는 역전파로 2개 훈련완료, 2개 훈련안됨이다.
```python
[(var.name, var.trainable) for var in model.layers[1].variables]
```

## 1.4 그레이디언트 클리핑 Gradient clipping

- gradient 폭주 문제 해결 방법
- 역전파시 일정 임곗값을 넘지 못하게 gradient를 잘라낸다

구현방법 코드 예시: clipvalue, clipnorm 매개변수 지정
```python
optimizer = keras.optimizers.SGD(clipvalue=1.0)
model.compile(loss="mse",optimizer=optimizer)
```
-1.0 ~ 1.0 범위로 그레이디언트 벡터의 모든 원소(편미분 값)를 클리핑한다.

# 2. 사전훈련된 층 재사용. 전이학습

큰 규모의 DNN을 처음부터 학습하기보다는 비슷한 유형의 문제를 처리한 신경망을 찾아서 해당 신경망의 하위층을 재사용하는 것이 좋다.    
이를 전이학습이라고 함.

- 보통 출력층을 바꿈 (출력 개수가 안맞을 확률 큼)
- 하위 은닉층(입력층에 가까운)이 상위 은닉층보다 더 유용하다
- 재사용할 층 갯수 선정 잘해야함

과정 
- 재사용층의 가중치 고정시키기
- 한 두개의 은닉층(출력층에 가까운) 가중치 변동하여 성능확인
- 훈련 데이터가 많으면 더 많은 은닉층 가중치 변경 가능(동결 해제)
- 동결 해제시 학습률을 줄이는게 좋다
- 재사용층의 적절한 개수를 찾을 때 까지 반복

Model Load

In [ ]:
model_A=keras.models.load_model("my_model_A.h5")

model_A 활용한 새로운 모델 만들기     
model_B 조작시 model_A가 영향을 받는 방법

In [ ]:
model_B_A = keras.models.Sequential(model_A.layers[:-1]) # 은닉층
model_B_A.add(keras.layers.Dense(1, activation="sigmoid")) # 출력층

model_B 조작시 model_A가 영향 안받는 방법, clone 하기

In [ ]:
model_A_clone=keras.models.clone_model(model_A)
model_A_clone.set_weights(model_A.get_weights())

은닉층의 부분만 (하위은닉층) 사용하기 , trainable=false 로 지정하여 동결을 해제하여 새로운 층을 학습시킬 수 있다.    
층의 동결 혹은 해제후에는 컴파일을 꼭 해주자.    
동결 해제후에는 학습률을 낮추는게 좋다. 

In [ ]:
for layer in model_B_A.layers[:-1]:
    layer.trainable=False
    
model_B_A.compile(loss="binary_crossentropy", optimizer="sgd", metrics=["accuracy"])

In [ ]:
history=model_B_A.fit(X_train_B, y_train_B, epochs=4, validation_data=(X_valid_B, y_valid_B))

for layer in model_B_A.layers[:-1]:
    layer.trainable=True
optimizer=keras.optimizers.SGD(lr=1e-4)
model_B_A.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])
history=model_B_A.fit(X_train_B,y_train_B, epochs=16, validation_data=(X_valid_B, y_valid_B))

- target class, random 초깃값을 바꾸면 좋던 성능도 낮아지기도 한다
- 논문 저자들은 가장 좋은 결과만 제출한다
- 전이학습은 작은 완전 연결 네트워크에서는 잘 동작하지 않는다
- 작은 네트워크 - 패턴 수를 적게 학습, 완전 연결 네트워크 - 특정 패턴 학습 이 원인 일수도 ...
- 전이 학습은 일반적인 특성-하위층을 감지하는 경향이 있는 심층 합성곱 신경망에서 잘 작동한다


## 2.2 비지도 사전훈련
비슷한 작업에 대한 전이 학습 모델을 찾기 힘들다면? 더 많은 레이블된 훈련 데이터 찾기! 그래도 힘들다면 비지도 사전훈련!
(오토인코더, 생성적 적대 신경망) , (오토인코더, GAN 판별자의 하위층 재사용, 맞는 출력층 추가) , (지도학습)

초기 층이 많이 모델 훈련하기 어려움 -> 탐욕적 층 단위 사전훈련 

# 3. 고속 옵티마이저
훈련 속도 높이는 방법
1. 연결 가중치에 좋은 초기화 전략
2. 좋은 활성화 함수
3. 배치 정규화 사용
4. 사전 훈련 네트워크 일부 재사용

그리고 5. 경사 하강법 optimizer 대신에, 더 빠른 옵티마이저 선택

모멘텀 최적화, 네스테로프 가속 경사, AdaGrad, RMSProp, Adam, Nadam 옵티마이저 ...

## 3.1 모멘텀 최적화
표준 경사 하강법 : 경사면에 따라 일정한 스텝으로 내려간다
모멘텀 최적화 : 느린 시작과 점점 빨라진다

표준 경사 하강법 : 이전 그레이디언트를 고려하지 않는다
모멘텀 최적화 : 이전 그레이디언트에 따라 가중치를 갱신한다 - 속도가 아닌 가속도 이용, 마찰 저항값으로 $\beta$로 모멘텀이 너무 커지는 것을 막는다. 0(높은 마찰 저항) ~ 1(마찰저항 없음).

표준 경사 하강법 : 입력값의 스케일이 다르면 가파른 경사는 빠르게 내려가고, 특정 좁고 긴 부분은 느리게 내려간다
모멘텀 최적화 : 골짜기를 따라 최적점에 빠르게 내려간다

```python
optimizer=keras.optimizers.SGD(lr=0.001, momentum=0.9)
```

### 3.2 네스테로프 가속 경사 Nesterov accelerated gradient
- 모멘텀 최적화의 변종
- 기본 모멘텀 최적화보다 거의 빠르다
- 모멘텀 벡터가 최적점을 향한 방향을 가리킨다는 가정
- 현재가 아닌 모멘텀의 방향으로 조금 앞선 위치에서 비용 함수의 gradient를 계산하다
```python
optimizer=keras.optimizers.SGD(lr=0.001, momentum=0.9, nesterov=True)
```

### 3.3 AdaGrad
한쪽이 길쭉한 그릇의 경우

경사 하강법 : 가장 가파른 경사를 따라 내려간다 - 정확한 방향성의 부재
AdaGrad : 가장 가파른 차원을 따라 gradient vector의 스케일을 감소하여 해결

과정
1. s += (gradient 제곱)    
만약 경사가 가파르다면 s값은 점점 커질 것이다
2. gradient vector / $\sqrt{s+\varepsilon}$ , $\varepsilon$은 분모가 0이 되지 않도록 하기위한 장치이다.

학습률을 감소시키지만, 경사가 가파른 곳에서 빠르게 감소된다. - 적응적 학습률 

단점 : 2차방정식 문제에 ㄱㅊ지만 일찍 학습을 멈추는 경우가 종종 있다. 학습률이 너무 감소되어 전역 최적점에 도착전에 멈추는 것이다.

### 3.4 RMSProp
AdaGrad 식에서 변형된 ... 가장 최근 반복된 gradient만 누적하여 사용한다

### 3.5 적응적 모멘트 추정 Adam, Nadam 최적화
Adam = 모멘텀 최적화 + RMSProp

Adam은    
- 모멘텀 최적화의 지난 Gradient의 지수 감소 평균
- RMSProp의 지난 Gradient 제곱의 지수 감소 평균     
을 이용한다.
- 시간에 따라 감쇠된 gradient $l_2$ 노름(제곱 합의 제곱근)으로 파라미터 업데이트의 스케일을 낮춘다.

```python
optimizer=keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999)
```

##### AdamMax
- s의 값을 가장 최댓값으로 갱신하고 adam의 차례 4번째를 실행하지 않는다.
- s값에 비례하여 gradient update scale을 낮춘다. 시간에 따라 감쇠된 gradient의 최댓값이다.
- Adam의 성능이 더 좋으나, Adam이 잘 작동하지 않는다면 대체할 수 있다.

##### Nadam
- Adam + 네스테로프 기법
- Adam 보다 빨리 수렴하기도


---

상위의 최적화 기법 = 1차 편미분에만 의존

2차 편미분 알고리즘은 심층 신경망에 적용하기 어렵다. 
Because 1개 output마다 $n^2$ 개의 2차 편미분을 진행해야 하기 때문이다. (1차 편미분은 $n$개0)


### 3.6 학습률 스케줄링
일정한 학습률을 사용하면    ...
너무 작으면 느리고 크면 발산해버리고 조금 크게 잡으면 처음에는 빠르게 진행하다가 발산해버리고 ...    
만약 자원의 한계가 있다면 완전히 수렴하기 전에 훈련이 멈추도록해야 한다

학습률을 상황에 따라 변동하도록 하면 ...    
큰 학습률로 시작해 학습 속도가 느려지면 학습률을 낮추도록 하는 방법    

- 거듭제곱 기반 스케줄링 
각 스텝마다 학습률을 감소한다. 처음에는 빠르게 감소하다가 점점 더 느리게 감소된다.    
$\eta$ -> $\eta$/2 -> $\eta$/4 -> ...

code : decay(s의 역수)
```python
keras.optimizers.SGD(lr=0.01, decay=1e-4)
```

- 지수 기반 스케줄링
스텝마다 10배씩 학습률이 감소한다. 

code 2가지
1.epoch을 이용한 학습률 function making
```python
def exponential_decay_fn(epoch):
    return 0.01*0.1**(epoch/20)
```
2.$\eta$와 s를 반환하는 closure 함수 
```python
def exponential_decay(lr0,s):
    def exponential_decay_fn(epoch):
        return lr0*0.1**(epoch/s)
    return exponential_decay_fn
exponential_decay_fn=dxponential_decay(lr0=0.01, s=20)
```

1혹은 2를 사용한 뒤 콜백을 만든다
```python
lr_scheduler=keras.callbacks.LearningRateSchedular(exponential_decay_fn)
history=model.fit(X_train_scaled,y_train, ,,,, callbakcs=[lr_scheduler])
```
LearningRateSchedular - 에포크시 learning_rate update.

- 구간별 고정 스케줄링
일정 에포크동안 일정 학습률을 사용하고, 다음 일정 에포크동안 더 작은 학습률을 사용한다.

- 성능 기반 스케줄링
매 N 스텝마다 검증 오차측정, 오차가 줄어들지 않으면 $\lambda$배 학습률 감소

- 1사이클 스케줄링
초기 학습률 $\eta_0$을 선형적으로 $\eta_1$까지 증가시킨다. 학습의 절반까지는 다시 $\eta_0$ 까지 줄이고,     
마지막 몇 번의 에포크는 소수점 자리로 학습률을 감소시킨다.    

Model Save - Optimizer, learning_rate 가 함께 저장된다.    
새로운 스케쥴 함수를 이용해서 학습을 이어갈 때도 문제없다.    
하지만 스케쥴 함수가 epoch 매개변수를 이용한다면 새로운 학습을 이어갈 때 epoch가 0에서 부터 다시 시작하므로 문제가 된다.
끝난 epoch에서 시작하려면 initial_epoch 매개변수를 지정해주는 방법이 있다.

# 4. 규제를 사용해 과대적합 피하기

## 4.1 $l_1$과 $l_2$ 규제
신경망의 연결 가중치 제한방법     
$l_2$ 규제 사용    
$l_1$ 규제 이용한 많은 가중치가 0인 희소 모델 생성

연결 가중치에 규제 강도 0.01 $l_2$ 규제 코드
```python
layer=keras.layers.Dense(100, activation="elu", 
                        kenel_initializer="he_normal", 
                        kernel_regularizer=keras.regularizers.12(0.01))
```
l2() 함수는 각 스텝에서의 규제 객체를 반환한다. 계산된 규제 손실은 최종 손실에 합산된다.

동일한 매개변수를 매번 적기보다 1. 반복문을 사용하거나 2. functools.partial()로 매개변수 값을 사용할 수 있다. 
```python
from functools import partial

RegularizedDense=partial(keras.layers.Dense,
                        activation="elu",
                        kernel_initializer="he_normal",
                        kernel_regularizer=keras.regularizers.l2(0.01))
model=keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28,28],
    RegularizedDense(300),
    RegularizedDense(100),
    RegularizedDense(10, activation="softmax", kernel_initalizer="glorot_uniform"))
])
```

## 4.2 Drop Out

- 매 스텝에서 각 뉴런은 임시적으로 드롭아웃될 확률 p를 가진다
- 보통 10% ~ 50%로 지정
- 순환 신경망 : 20~30%, 합성곱 신경망 40~50%
- 몇 개의 입력 뉴런에 의지할 수 없게된다. 
- 드롭아웃을 사용하면 하나의 뉴런이 두 배 많은 입력 뉴런과 연결된다 -> 각 뉴런의 연결 가중치에 0.5를 곱해서 잘 작동하도록 해준다

```python
model=... ,
        keras.layers.Dropout(rate=0.2),
        ... ,
        keras.layers.Dropout(rate=0.2),
        keras.layers.Dense(10, activation="softmax")
        ...
```
- 과대적합시 드롭아웃의 비율을 늘린다
- 층이 클 때는 드롭아웃 비율을 늘리고 작은 층은 드롭아웃 비율을 줄인다
- 은닉층이 너무 많다면(모두 드롭아웃을 적용하기 부담스럽다면) 마지막 은닉층 뒤에만 드롭아웃을 사용해본다


## 4.3 몬테 카를로 드롭아웃
드롭아웃 모델의 성능을 재훈련 하지않고도 높여주고 더 정확한 불확실성 추정을 제공한다.

샘플의 수가 높을수록 정확도가 높아질수는 있지만 예측 시간도 늘어난다.    
일정 샘플 수가 넘어서면 성능이 크게 향상되지 않음. 균형점을 찾아라.

처음부터 MCDropout층을 만들거나 ... 

BatchNormalization과 같은 랜덤으로 지정되는 층이 있다면 아래와 같은 방법을 사용한다
Dropout한 모델의 경우 Dropout을 MCDropout으로 만드는 방법    
기존 모델의 가중치를 복사할 수 있다
```python
class MCDropout(keras.layers.Dropout):
    def call(self, inputs):
        return super().call(inputs, training=True)
```
Dropout 층을 상속
call() 메서드 override
training=True로 

## 4.4 맥스-노름 규제
각각의 뉴런에 대해 입력의 연결 가중치의 $l_2$값이 맥스-노름 하이퍼파라미터 값 이하가 되도록 제한한다.    
맥스-노름 하이퍼파라미터값을 줄이면 규제의 양이 증가하며, 과대적합을 감소시킨다.    
배치 정규화를 사용하지 않았을 때 큰 도움이 된다.    

max_norm()의 객체 kernel_constraint 매개변수 지정
```python
keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal",
                    kernel_constraint=keras.constraints.max_norm(1.))
```

max_norm()의 axis 매개변수 : 0 이 default. (샘플 개수, 뉴런개수)    

axis=0 : 각 뉴런의 가중치 벡터에 독립적으로 적용

# 5. 궁합 정리
기본 DNN 설정
- 커널 초기화 : He 초기화
- 활성화 함수 : ELU
- 정규화 : 얕은 신경 (무), 깊은 신경망 (배치 정규화)
- 규제 : 조기 종료 (+ $l_2$)
- optimizer : 모멘텀 최적화 (or RMSProp or Nadam)
- Learning_rate : 1 사이클

자기 정규화를 위한 DNN 설정
- 커널 초기화 : 르쿤 초기화
- 활성화 함수 : SELU
- 정규화 : 없음
- 규제 : + alpha Dropout
- optimizer : 모멘텀 최적화 (or RMSProp or Nadam)
- Learning_rate : 1 사이클

희소 모델 
- $l_1$ 규제
- 텐서플로 모델 최적화 툴킷 사용
- 자기 정규화를 깨뜨리므로 기본 DNN 설정 써야함

빠른 모델
- 은닉층의 개수 줄이기
- 배치 정규화 층을 이전 층에 합치기
- LeakyReLU, ReLU 빠른 활성화 함수 사용
- 희소 모델
- 부동소수점 정밀도를 32bit -> 16 -> 8bit로